# Intro to Data Science - Final Project 

##### Copyright 2022, Group 4


#1-Importing the libraries:

In [ ]:
install.packages('arules')
install.packages('arulesViz')
install.packages('MASS')
install.packages('maps')
install.packages('ggplot2')
install.packages('ggmap')
install.packages('mapproj')
install.packages('imputeTS')
install.packages('caret')
install.packages('arulesViz')
install.packages('arules')
install.packages('kernlab')
library(tidyverse)
library(MASS)
library(imputeTS)
library(ggplot2)
library(maps)
library(ggmap)
library(mapproj)
library(kernlab)
library(caret)
library(arules)
library(arulesViz)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘foreach’, ‘zoo’, ‘tweenr’, ‘polyclip’, ‘RcppEigen’, ‘gridExtra’, ‘RcppArmadillo’, ‘later’, ‘TSP’, ‘qap’, ‘gclus’, ‘ca’, ‘registry’, ‘lmtest’, ‘Rcpp’, ‘ggforce’, ‘ggrepel’, ‘viridis’, ‘tidygraph’, ‘graphlayouts’, ‘htmlwidgets’, ‘crosstalk’, ‘promises’, ‘lazyeval’, ‘seriation’, ‘vcd’, ‘igraph’, ‘scatterplot3d’, ‘ggraph’, ‘DT’, ‘plotly’, ‘visNetwork’




#2-Data Source, and Data Cleaning

###2.1--Storing the data in df dataset


In [ ]:
df<- read_csv('https://intro-datascience.s3.us-east-2.amazonaws.com/HMO_data.csv')
head(df)

###2.2-Converting the data type to factor:


In [ ]:
df <- mutate_if(df,is.character, factor)
df$children <- factor(df$children)
head(df)

###2.3-Checking for NA values:


In [ ]:
cat('Number of null values in age column: ', sum(is.na(df$age)))
cat("\n")
cat('Number of null values in bmi column: ', sum(is.na(df$bmi)))
cat("\n")
cat('Number of null values in children column: ', sum(is.na(df$children)))
cat("\n")
cat('Number of null values in cost column: ', sum(is.na(df$cost)))
cat("\n")
cat('Number of null values in education_level column: ', sum(is.na(df$education_level)))
cat("\n")
cat('Number of null values in exercise column: ', sum(is.na(df$exercise)))
cat("\n")
cat('Number of null values in gender column: ', sum(is.na(df$gender)))
cat("\n")
cat('Number of null values in hypertension column: ', sum(is.na(df$hypertension)))
cat("\n")
cat('Number of null values in location column: ', sum(is.na(df$location)))
cat("\n")
cat('Number of null values in location_type column: ', sum(is.na(df$location_type)))
cat("\n")
cat('Number of null values in married column: ', sum(is.na(df$married)))
cat("\n")
cat('Number of null values in smoker column: ', sum(is.na(df$smoker)))
cat("\n")
cat('Number of null values in yearly_physical column: ', sum(is.na(df$yearly_physical)))

###2.4-Interpolating the Null values: 

In [ ]:
df$bmi <- na_interpolation(df$bmi)
df$hypertension <- as.numeric(df$hypertension)
df$hypertension <- na_interpolation(df$hypertension)
df$hypertension <- as.factor(df$hypertension)
#removing the rows with interpolation of 0.5
df <- df%>%filter(hypertension != 0.5)
df$hypertension <- droplevels(df$hypertension)
cat('Number of null values in hypertension column: ', sum(is.na(df$hypertension)))
cat("\n")
cat('Number of null values in bmi column: ', sum(is.na(df$bmi)))
cat("\n")
head(df)

###2.5-Adding the Expensive Column:

In [ ]:
# df$expensive <- ""
# df$expensive[df$cost < 5000] <- 0
# df$expensive <- as.factor(df$expensive)
df$expensive <- "TRUE"
df$expensive[df$cost < 5000] <- "FALSE"
df$expensive <- as.factor(df$expensive)
head(df)

###2.6-Get the number of expensive and inexpensive obsservation

In [ ]:
print('Table of expensive:')
table(df$expensive)
# from this output we can say that the there are 1793 expensive and 5759 inexpensive

###2.7-Get a copy of main dataset and remove the expensive column:


In [ ]:
# df_c  contains the cost column and not the expensive column for training the linear regression model.
df_c <- df 
df_c <- df_c[-15]
head(df_c)

###2.8-Get a copy of main dataset and remove the cost column:


In [ ]:
#df_e contains the expensive column and not the cost column for training the Support Vector Machine(SVM), Logistic Regression, and Association Rules mode.
df_e <- df 
df_e <- df_e[-14]
head(df_e)

#3- Important Variables:

###3.0-Creating expensive & none-expensive subsets:

In [ ]:
#create two subset of the data regarding expensiveness
df_exp <- df[df$expensive==TRUE,]
df_cheap <- df[df$expensive==FALSE,]

###3.1- age:

In [ ]:
ggplot(data=df) + aes(x=age, y=cost) + geom_point(color="deepskyblue2") + geom_smooth(method="lm") + coord_cartesian( xlim = NULL, ylim = NULL, expand = TRUE, default = FALSE, clip = "on")
#As age increases medical costs go up.

###3.2- bmi:

In [ ]:
ggplot(data=df) + aes(x=bmi, y=cost) + geom_point(color="deepskyblue2") + geom_smooth(method="lm") + coord_cartesian( xlim = NULL, ylim = NULL, expand = TRUE, default = FALSE, clip = "on")
summary(lm(cost~bmi, data=df))
#As bmi increases medical costs go up.

###3.3- children:

In [ ]:
ggplot(data=df) + aes(x=factor(children), y=cost,fill=children) + geom_bar(stat='identity') 
summary(lm(cost~children, data=df))
#People who have no children are more expensive.

###3.4- smoker:

In [ ]:
ggplot(data=df_exp) + aes(x=factor(smoker), y=cost, fill=smoker) + geom_bar(stat='identity')
#People who smoke spend more money on healthcare than those who dont smoke.

###3.5- location:

In [ ]:
ggplot(data=df) + aes(x=factor(location), y=cost, fill=location) + geom_bar(stat='identity')
#Pennsylvania has the highest cost.

###3.6- location_type:

In [ ]:
ggplot(data=df) + aes(x=factor(location_type), y=cost, fill=location_type) + geom_bar(stat='identity')
#People who live in urban areas spend more on healtcare than those who live in cities

###3.7- Education_level:

In [ ]:
ggplot(data=df) + aes(x=factor(education_level), y=cost, fill=education_level) + geom_bar(stat='identity')
#People who have a bachelor degree are more expensive.

###3.8- yearly_physical:

In [ ]:
ggplot(data=df) + aes(x=factor(yearly_physical), y=cost, fill=yearly_physical) + geom_bar(stat='identity')
#People who are not active spend more on healthcare than those who are on average.

###3.9- Exercise:

In [ ]:
ggplot(data=df) + aes(x=factor(exercise), y=cost, fill=exercise) + geom_bar(stat='identity')
#People who dont exercise spend more on healthcare than those who do on average.

###3.10- Married:

In [ ]:
ggplot(data=df) + aes(x=factor(married), y=cost, fill=married) + geom_bar(stat='identity')
#Married people are more expensive than unmarried

###3.11- hypertension:

In [ ]:
ggplot(data=df) + aes(x=factor(hypertension), y=cost, fill=hypertension) + geom_bar(stat='identity')

###3.12- Gender:

In [ ]:
ggplot(data=df) + aes(x=factor(gender), y=cost, fill=gender) + geom_bar(stat='identity')
#Males are more expensive than females.

###3.13-Creating the Table of Categorical variables:


In [ ]:
print('Table of education_level:')
table(df$education_level)
print('Table of Children:')
table(df$children)
print('Table of hypertension:')
table(df$hypertension)
print('Table of exercise:')
table(df$exercise)
print('Table of expensive:')
table(df$expensive)
print('Table of gender:')
table(df$gender)
print('Table of location:')
table(df$location)
print('Table of location_type:')
table(df$location_type)
print('Table of married:')
table(df$married)
print('Table of smoker:')
table(df$smoker)
print('Table of yearly_physical:')
table(df$yearly_physical)

None of observed variables have to be normal in linear regression analysis, which includes t-test and ANOVA. The errors after modeling, however, should be normal to draw a valid conclusion by hypothesis testing


###3.13-Creating a Geographic Map:

In [ ]:
dfmap <- df 
dfmap$region <- tolower(dfmap$location)

dfmap$expensive <- as.numeric(dfmap$expensive)

states <- map_data("state")
map.dfmap <- merge(states,dfmap, by="region", all.x=T)
map.dfmap <- map.dfmap[order(map.dfmap$order),]
ggplot(map.dfmap, aes(x=long,y=lat,group=group))+
  geom_polygon(aes(fill=expensive))+
  geom_path()+ 
  scale_fill_gradientn(colours=rev(heat.colors(10)),na.value="grey90")+
  coord_map()

dfmap$expensive <- as.factor(dfmap$expensive)

#4-Linear Regression:

###4.1-Generate a list of cases to include in the training data:

In [ ]:
set.seed(111)
trainListlm <- createDataPartition(y=df$cost, p=.70, list=FALSE)

###4.2-Create train and test data set:

In [ ]:
trainDatalm <- df [trainListlm,]
testDatalm <-  df [-trainListlm,]

###4.3-What are the significant variable:

In [ ]:
lmodelFinalsignif <- lm(cost ~ . ,data=trainDatalm)
summary(lmodelFinalsignif)

###4.4-Constructing the linear model:


In [ ]:
# # lmodel1 <- lm(cost ~.,data=sorted_trainDataLM1)
# # summary(lmodel1)

# lmodel1 <- lm(cost ~ age+ bmi+ children+ smoker+ location+ exercise + yearly_physical+ hypertension + married ,data=df)
# summary(lmodel1)
lmodelFinal <- lm(cost ~ age+ bmi+ children+ smoker+ location+ exercise+ hypertension ,data=trainDatalm)
summary(lmodelFinal)

###4.5-Prediction for the test data set:

In [ ]:
# PredtestData<- data.frame(predict(lmodel1, sorted_testDataLM1))
# PredtestData$Actual_value <- sorted_testDataLM1$cost
# head(PredtestData)
predictValues <- predict(lmodelFinal, newdata=testDatalm)
PV <- data.frame(predictValues)
head(PV)

###4.6-Add expensive column:

In [ ]:
PV$expensive <- 1
PV$expensive[PV$predictValues < 5000] <- 0
PV$expensive <- factor(PV$expensive)
head(PV)

###4.7-Calculate the confusion matrix:

In [ ]:
table(PV$expensive,testDatalm$expensive)

###4.8-Calculate the accuracy:

In [ ]:
sum(diag(table(PV$expensive, testDatalm$expensive)))/sum(table(PV$expensive, testDatalm$expensive))

###4.9- Sensitivity:

In [ ]:
415/(415+123)

###4.10-Import and clean the sample data:

In [ ]:
sampleData <- read_csv("/content/HMO_TEST_data_sample.csv")
sampleData <- mutate_if(sampleData,is.character, factor)
sampleData$children <- as.factor(sampleData$children)
sampleData$hypertension <- as.factor(sampleData$hypertension)
head(sampleData)

###4.11-Training the linear model with the whole data set:

In [ ]:
lmodelFinal2 <- lm(cost ~ age+ bmi+ children+ smoker+ location+ exercise+ hypertension ,data=df)
summary(lmodelFinal2)

###4.12-Prediction for sample data:

In [ ]:
predFinal2 <- predict(lmodelFinal2, newdata=sampleData)
PV2 <- data.frame(predFinal2)
PV2

###4.13-Change the format of prediction data:

In [ ]:
#Add index:
PV2$X <- sampleData$X
#Add expensive column:
PV2$expensive <- 1
PV2$expensive[PV2$predFinal2 < 5000] <- 0
PV2$expensive <- factor(PV2$expensive)
#Changing the expensive column values from 0/1 to FALSE/TRUE
PV3 <- PV2
PV3$expensive <- as.character(PV3$expensive)
PV3$expensive[PV3$expensive == '1'] <- 'TRUE'
PV3$expensive[PV3$expensive == '0'] <- 'FALSE'
PV3$expensive <- as.factor(PV3$expensive)
PV3 <- (PV3[, -1])
head(PV3)

###4.14-Importing the sample solution:

In [ ]:
sampleSolution <- read.csv("/content/HMO_TEST_data_sample_solution.csv")
sampleSolution

###4.15- Calculate the confusion matrix:



In [ ]:
table(sampleSolution$expensive, PV3$expensive)

###4.15-Calculating the accuracy for the sample data:

In [ ]:
sum(diag(table(sampleSolution$expensive, PV3$expensive)))/sum(table(sampleSolution$expensive, PV3$expensive))

###4.16-Calculating the sensitivity:

In [ ]:
4/(4+5)

#5-SVM:

###5.1-Generate a list of cases to include in the training data:

In [ ]:
set.seed(111)
trainList <- createDataPartition(y=df_e$expensive, p=.70, list=FALSE)

###5.2-Create train and test data set:

In [ ]:
trainData <- df_e [trainList,]
testData <-  df_e [-trainList,]

###5.3-Import and merge the sample data and sample solution:

In [ ]:
#Import the sample data:
sampleData <- read_csv("/content/HMO_TEST_data_sample.csv")
#Importing the sample solution:
sampleSolution <- read.csv("/content/HMO_TEST_data_sample_solution.csv")
#Joining the samples data and sample solution:
SampleMergedsvm <- merge(sampleSolution, sampleData, by = "X")
#Changing the sequence of columns:
SampleMergedsvm <- SampleMergedsvm[, c(1,3,4,5,6,7,8,9,10,11,12,13,14,2)]
SampleMergedsvm <- mutate_if(SampleMergedsvm,is.character, factor)
SampleMergedsvm$children <- as.factor(SampleMergedsvm$children)
SampleMergedsvm$hypertension <- as.factor(SampleMergedsvm$hypertension)
#Adding one row from the main data set to adjust the levels
Sample_additionsvm <- df_e%>%filter(children == 4)%>%filter(location == "RHODE ISLAND")
Sample_additionsvm <-Sample_additionsvm[1,]
SampleMergedsvm <- rbind(SampleMergedsvm, Sample_additionsvm)
#changing the variable expensive type
SampleMergedsvm$expensive <- as.factor(SampleMergedsvm$expensive)
#Explore the sample data 
head(SampleMergedsvm)

###5.4-Training the SVM model with train data:

In [ ]:
svmModel <- ksvm(expensive ~ smoker+ age+ bmi+ exercise + children, data = trainData, C = 5, cross = 3, prob.model = TRUE)
#Investigate the SVM model:
svmModel

# smoker+ age+ bmi+ exercise : Sensitivity : 0.9774, Accuracy : 0.8807
# smoker+ age+ bmi+ exercise + children : Sensitivity : 0.9751, Accuracy : 0.8838
# smoker+ age+ bmi+ exercise+ married, Sensitivity : 0.9768, Accuracy : 0.8812
# age+ bmi+ children+ smoker+ location+ exercise+ hypertension : Sensitivity : 0.9710, Accuracy : 0.8772
# . : Sensitivity : 0.9722 , Accuracy : 0.8790 

###5.5-Prediction for the test data set using the SVM model:

In [ ]:
predSVM <- predict(svmModel, newdata = testData, type = "response")

###5.6-Calculate	the confusion	matrix and	the accuracy	:

In [ ]:
confusionMatrix(predSVM, testData$expensive)

###5.7-Training the SVM model with the whole data set:

In [ ]:
svmModelFinal <- ksvm(expensive ~smoker+ age+ bmi+ exercise + children , data = df_e, C = 5, cross = 3, prob.model = TRUE)
svmModelFinal 

# smoker+ age+ bmi+ exercise : Sensitivity : 0.7500, Accuracy : 0.619
# smoker+ age+ bmi+ exercise + children : Sensitivity : 0.7500, Accuracy : 0.619
# age+ bmi+ children+ smoker+ location+ exercise+ hypertension : Sensitivity : 0.7500, Accuracy : 0.5714 
# smoker+ age+ bmi+ exercise+ married, Sensitivity : 0.7500, Accuracy : 0.5714
# . : Sensitivity : 0.6667 , Accuracy : 0.5238 

###5.8-Prediction for sample data:

In [ ]:
predSVMSample <- predict(svmModelFinal, newdata = SampleMergedsvm, type = "response")

###5.9-Calculate	the confusion	matrix and	the accuracy	:

In [ ]:
confusionMatrix(predSVMSample, SampleMergedsvm$expensive)

#6-Association Rules:


###6.1-Descritize the continuous variables and remove the index column:

In [ ]:
dfassoc <- df
dfassoc$cost <- discretize(dfassoc$cost, method = "interval", breaks = 3, labels = c("small", "medium", "large"))
dfassoc$bmi <- discretize(dfassoc$bmi, method = "interval", breaks = 3, labels = c("small", "medium", "large"))
dfassoc$age <- discretize(dfassoc$age, method = "interval", breaks = 3, labels = c("young", "middle", "old"))
dfassoc <- dfassoc[,-1]
head(dfassoc, 10)


###6.2-One hotencoding the data set:

In [ ]:
dftransactions <- as(dfassoc, 'transactions')

###6.3-Generate a list of item frequencies

In [ ]:
dfIF <- itemFrequency(dftransactions)
str(dfIF)

###6.4-Show the most	and	least	frequent features:

In [ ]:
dfIF <- sort(-dfIF)
head(dfIF,14)
tail(dfIF,14)

###6.5-Create	a	frequency	plot:

In [ ]:
itemFrequencyPlot(dftransactions, topN=20)

###6.6-Create	a	cross	table:

In [ ]:
ct	<-	crossTable(dftransactions,	sort=TRUE)
ct[1:14,	1:14]

###6.7-Create association rules:

In [ ]:
assocRules <- apriori(dftransactions,
 parameter=list(supp=0.5, conf=0.88),
 control=list(verbose=F),
 appearance=list(default="lhs",rhs=("expensive=FALSE")))

###6.8-Invesitigate the rules:

In [ ]:
assocRules
inspect(assocRules)

#7- Logistic Regression:

###7.1-Training the Logistic model:

In [ ]:
Logistic_Model = glm(expensive ~ ., data=trainData, family=binomial)
summary(Logistic_Model)

###7.2-Prediction for the trainging data using logistic model:

In [ ]:
LogiProb0 = predict(Logistic_Model, newdata = testData, type="response")
LogiPred0 = rep("FALSE", dim(df_e)[1])
LogiPred0[LogiProb0 > .5] = "TRUE"

###7.3-Calculating the confusion matrix for the logistic model:

In [ ]:
table(LogiPred0, df_e$expensive)

###7.4- Calculating the accuracy:

In [ ]:
sum(diag(table(LogiPred0, df_e$expensive)))/sum(table(LogiPred0, df_e$expensive))

###7.5- Calculating the sensitivity:

In [ ]:
356/(356+1437)

###7.6-Importing and cleaning the sample data:


In [ ]:
#Import the sample data:
sampleData <- read_csv("/content/HMO_TEST_data_sample.csv")
#Importing the sample solution:
sampleSolution <- read.csv("/content/HMO_TEST_data_sample_solution.csv")
#Joining the samples data and sample solution:
SampleMerged <- merge(sampleSolution, sampleData, by = "X")
#Changing the sequence of columns:
SampleMerged <- SampleMerged[, c(1,3,4,5,6,7,8,9,10,11,12,13,14,2)]
SampleMerged <- mutate_if(SampleMerged,is.character, factor)
SampleMerged$children <- as.factor(SampleMerged$children)
SampleMerged$hypertension <- as.factor(SampleMerged$hypertension)
#Adding one row from the main data set to adjust the levels
Sample_addition <- df_e%>%filter(children == 4)%>%filter(location == "RHODE ISLAND")
Sample_addition <-Sample_addition[1,]
SampleMerged <- rbind(SampleMerged, Sample_addition)
#Explore the sample data 
#SampleMerged

###7.7-Training the Logistic model with the whole data set:

In [ ]:
Logistic_Model_Final = glm(expensive ~ ., data=df_e, family=binomial)
summary(Logistic_Model_Final)

###7.8-Prediction for the sample data using logistic model:

In [ ]:
LogiProb = predict(Logistic_Model_Final, newdata = SampleMerged, type="response")
LogiPred = rep("FALSE", dim(df_e)[1])
LogiPred[LogiProb > .5] = "TRUE"

###7.9-Calculating the accuracy for the sample data:

In [ ]:
table(LogiPred, df_e$expensive)

###7.10- Calculating the accuracy:

In [ ]:
sum(diag(table(LogiPred, df_e$expensive)))/sum(table(LogiPred, df_e$expensive))

###7.11-Calculating th sensitivity:

In [ ]:
578/(578+1215)